<a href="https://colab.research.google.com/github/SihyeonJeon/Modulation-aware-key-estimator/blob/main/Mod_aware_KE_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#####setting

In [ ]:
# #@markdown #Gdrive connection
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@markdown ##Installation
print("Installing..")
!git clone https://github.com/SihyeonJeon/Modulation-aware-key-estimator.git
%cd Modulation-aware-key-estimator
!pip install -r requirements.txt

In [ ]:
#@markdown ##restart
import os
os.kill(os.getpid(), 9)

In [ ]:
# @title Function

import sys
sys.path.append('/content/Modulation-aware-key-estimator/app')

from model_loader import load_model
from utils import finalize_downloaded_wav
import subprocess
import uuid
import os
from pathlib import Path
import yt_dlp
import torch
import torchaudio
import os
from IPython.display import Audio
import numpy as np
import torch.nn.functional as F
from utils import preprocess_waveform, compute_chromagram, compute_hpcp, pitch_shift_segments
from model_loader import load_model
from inference import run_inference


# 모델 로드
model_checkpoint_path = "/content/drive/MyDrive/a2/key_model.pt"
model = load_model(model_checkpoint_path)



# 유튜브 다운로드
def download_youtube_audio(youtube_url, output_dir="/content"):
    os.makedirs(output_dir, exist_ok=True)
    unique_id = uuid.uuid4().hex[:8]
    output_template = f"{output_dir}/%(title)s_{unique_id}.%(ext)s"
    download_command = [
        "yt-dlp",
        "-x", "--audio-format", "wav",
        "--output", output_template,
        "--encoding", "utf-8",
        "--cookies", "/content/Modulation-aware-key-estimator/cookies.txt",
        "--geo-bypass",
        youtube_url
    ]
    subprocess.run(download_command, check=True)
    return finalize_downloaded_wav(output_dir)

def standardize_key(key):
    key = key.strip().lower()
    flat_to_sharp = {
        'db': 'c#',
        'eb': 'd#',
        'gb': 'f#',
        'ab': 'g#',
        'bb': 'a#'
    }
    # 매핑 적용
    return flat_to_sharp.get(key, key)

target_key_list = ['c', 'c#', 'd', 'd#', 'e', 'f', 'f#', 'g', 'g#', 'a', 'a#', 'b']



# 🎵 유튜브 오디오 키 변환

전조가 있는 음악도 모든 구간이 일관된 키로 이동합니다.


### 입력값

1. 좋아하는 음악의 유튜브 링크(10분 미만 권장)
<br>
2. 변환할 key (대소문자 상관없음, # 또는 b 사용)

In [ ]:

#@markdown ## Music Processing
#@markdown ---
youtube_url = "https://www.youtube.com/watch?v=C_aaRUR738g" #@param {type:"string"}
to_key = "c" #@param {type:"string"}

#@markdown ---

# 입력된 키 표준화
to_key_standard = standardize_key(to_key)
try:
    target_key_index = target_key_list.index(to_key_standard)
except ValueError:
    raise ValueError(f"타겟 키 '{to_key_standard}'가 유효하지 않습니다.")

print("처리 중...")

# 유튜브 다운로드 및 모델 실행
wav_path = download_youtube_audio(youtube_url)
result = run_inference(wav_path, model, target_key_index=target_key_index)

print("결과:", result)

print(f"Original WAV: {wav_path}")
from IPython.display import Audio
display(Audio(wav_path))

shifted_wav_path = result['shifted_wav_path']
print(f"Pitch-shifted WAV: {shifted_wav_path}")
display(Audio(shifted_wav_path))
